In [3]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding, EarlyStoppingCallback

In [4]:
df = pd.read_excel('/kaggle/input/students-anxiety-and-depression-dataset/dataset.xlsx')
df = df.dropna(subset=["text", "label"])

texts = df["text"].astype(str).tolist()
labels = df["label"].astype(int).tolist()

In [5]:
df.head()

,text,label
0,oh my gosh,1.0
1,"trouble sleeping, confused mind, restless hear...",1.0
2,"All wrong, back off dear, forward doubt. Stay ...",1.0
3,I've shifted my focus to something else but I'...,1.0
4,"I'm restless and restless, it's been a month n...",1.0


In [6]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2).to("cuda")  # ✅ Move to GPU

class AnxietyDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

train_dataset = AnxietyDataset(train_texts, train_labels, tokenizer)
val_dataset = AnxietyDataset(val_texts, val_labels, tokenizer)

In [8]:
# Training config
training_args = TrainingArguments(
    output_dir="./bert_anxiety",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=DataCollatorWithPadding(tokenizer), 
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  
)

In [9]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.105600,0.046259
2,0.006400,0.042624
3,0.000900,0.053422


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=1047, training_loss=0.04916803249128611, metrics={'train_runtime': 331.2373, 'train_samples_per_second': 50.502, 'train_steps_per_second': 3.161, 'total_flos': 1100330433515520.0, 'train_loss': 0.04916803249128611, 'epoch': 3.0})

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

max_words = 5000
max_len = 100

tokenizer_lstm = Tokenizer()
tokenizer_lstm.fit_on_texts(texts)
X = tokenizer_lstm.texts_to_sequences(texts)
X = pad_sequences(X, maxlen=max_len)

vocab_size = len(tokenizer_lstm.word_index) + 1

X_train, X_val, y_train, y_val = train_test_split(X, labels, test_size=0.2, random_state=42)


model_lstm = Sequential()
model_lstm.add(Embedding(input_dim=vocab_size, output_dim=64, input_length=max_len))
model_lstm.add(LSTM(64, implementation=1))  # ✅ Forces CPU-compatible version
model_lstm.add(Dense(1, activation='sigmoid'))

model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
I0000 00:00:1746356572.346307      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 11494 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1746356572.346975      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 12632 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [14]:
import numpy as np

X_train = np.array(X_train).astype("int32")
X_val = np.array(X_val).astype("int32")
y_train = np.array(y_train).astype("float32")
y_val = np.array(y_val).astype("float32")


In [15]:
history = model_lstm.fit(
    X_train, y_train,
    epochs=5,
    batch_size=16,
    validation_data=(X_val, y_val)
)

Epoch 1/5


I0000 00:00:1746356641.501618    3094 cuda_dnn.cc:529] Loaded cuDNN version 90300


349/349 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.8885 - loss: 0.3457 - val_accuracy: 0.9828 - val_loss: 0.0674
Epoch 2/5
349/349 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9875 - loss: 0.0479 - val_accuracy: 0.9885 - val_loss: 0.0484
Epoch 3/5
349/349 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9945 - loss: 0.0210 - val_accuracy: 0.9857 - val_loss: 0.0532
Epoch 4/5
349/349 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9973 - loss: 0.0100 - val_accuracy: 0.9842 - val_loss: 0.0662
Epoch 5/5
349/349 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9969 - loss: 0.0124 - val_accuracy: 0.9878 - val_loss: 0.0519


In [17]:
from sklearn.metrics import classification_report, confusion_matrix
import torch

# Predict
pred_output = trainer.predict(val_dataset)
pred_logits = torch.tensor(pred_output.predictions)
preds = torch.argmax(pred_logits, dim=1)
true_labels = torch.tensor(pred_output.label_ids)

# Evaluation
print(" BERT Classification Report:")
print(classification_report(true_labels, preds, digits=3))

print(" Confusion Matrix:")
print(confusion_matrix(true_labels, preds))


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


 BERT Classification Report:
              precision    recall  f1-score   support

           0      0.997     0.993     0.995      1235
           1      0.945     0.975     0.960       159

    accuracy                          0.991      1394
   macro avg      0.971     0.984     0.977      1394
weighted avg      0.991     0.991     0.991      1394

 Confusion Matrix:
[[1226    9]
 [   4  155]]


In [18]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Get predictions (probabilities → binary)
y_pred = (model_lstm.predict(X_val) > 0.5).astype("int32")

# Print classification report
print("\n LSTM Classification Report:")
print(classification_report(y_val, y_pred))

# Print confusion matrix
print(" Confusion Matrix:")
print(confusion_matrix(y_val, y_pred))


44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

 LSTM Classification Report:
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99      1235
         1.0       0.96      0.94      0.95       159

    accuracy                           0.99      1394
   macro avg       0.97      0.97      0.97      1394
weighted avg       0.99      0.99      0.99      1394

 Confusion Matrix:
[[1228    7]
 [  10  149]]
